# Fase 2
NOTA:
- In questa fase, le immagini da inserire nel database saranno etichettate come image-X-Y- Z.png, dove
    - X ∈ {cc, con, detail, emboss, jitter, neg, noise1, noise2, original, poster, rot, smooth, stipple} denota il tipo dell’immagine;
    - 1≤Y≤40denota ilsubjectID,and
    - 1 ≤ Z ≤ 10 denota l’ ID del campione dell’immagine.
    

- I task in questa fase riguardano i tre modelli di feature e le misure di distanza/similarita’ sviluppate nella precedente.
 
- Potete usare le librerie esistenti per la decomposizione LDA.
- Potete usare le librerie esistenti per l’estrazione di autovalori e autovettori.

In [10]:
from ipynb.fs.full.task1_new import *
import numpy as np
from operator import itemgetter

from PIL import Image
from numpy import asarray
from scipy.stats import skew

# load and display an image with Matplotlib
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from scipy.linalg import svd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
import re
from sklearn.utils.extmath import randomized_svd
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

import networkx as nx



In [2]:
# import OS module
import os

# Get the list of all files and directories
path = "./secondaparte/images/"
dir_list = os.listdir(path) # array di stringhe contenente i nomi dei file
 
dir_list.sort()
dir_list.remove('.DS_Store')

print(len(dir_list))

4787


In [3]:
def createDictionaryAndWriteOnFile(matrix, filtered_dir_list, passedParameter, strategyName, modelName):
    matrixtrasposta = matrix.T

    dictionaryFeaturesLatenti = {}
    for i in range(len(matrixtrasposta)):
        sequenzaFeatureLatente = []
        for j in range(len(matrixtrasposta[i])):
            splittedString = re.split('-', filtered_dir_list[j])
            pair = ()

            if passedParameter == 'X':
                pair = (splittedString[2], matrixtrasposta[i][j])
            elif passedParameter == 'Y':
                pair = (splittedString[1], matrixtrasposta[i][j])
            
            sequenzaFeatureLatente.append(pair)
        
        sequenzaOrdinata = sorted(sequenzaFeatureLatente, key=lambda x: x[1], reverse=True)
        dictionaryFeaturesLatenti['Feature' + str(i)] = sequenzaOrdinata

    #print(dictionaryFeaturesLatenti)

    fileName = ''

    if passedParameter == 'X':
        fileName = 'task1_' + modelName + '_' + strategyName + '_coppie_soggetto-peso.txt'
    elif passedParameter == 'Y':
        fileName = 'task2_' + modelName + '_' + strategyName + '_coppie_tipo-peso.txt'

    # controlliamo se il file esiste o no
    file = open('./outputfilesecondaparte/' + fileName,'w+')

    with open('./outputfilesecondaparte/' + fileName, 'w') as convert_file:
        convert_file.write(json.dumps(dictionaryFeaturesLatenti, indent=4, sort_keys=True))

In [4]:
def createDictionaryAndWriteOnFileTask34(matrix, filtered_dir_list, passedParameter, strategyName, modelName):
    matrixtrasposta = matrix.T

    print(filtered_dir_list)
    print("LEN LIST", len(filtered_dir_list))
    print("len(matrixtrasposta)", len(matrixtrasposta))
    print("len(matrixtrasposta[0])", len(matrixtrasposta[0]))

    dictionaryFeaturesLatenti = {}
    for i in range(len(matrixtrasposta)):
        sequenzaFeatureLatente = []
        for j in range(len(matrixtrasposta[i])):
            pair = ()

            if passedParameter == 'soggetto':
                pair = (filtered_dir_list[j], matrixtrasposta[i][j])
            elif passedParameter == 'tipo':
                pair = (filtered_dir_list[j], matrixtrasposta[i][j])
            
            sequenzaFeatureLatente.append(pair)
        
        sequenzaOrdinata = sorted(sequenzaFeatureLatente, key=lambda x: x[1], reverse=True)
        dictionaryFeaturesLatenti['Feature' + str(i)] = sequenzaOrdinata

    print("DOPO IL FOR")

    #print(dictionaryFeaturesLatenti)

    fileName = ''

    if passedParameter == 'soggetto':
        fileName = 'task4_' + modelName + '_' + strategyName + '_coppie_soggetto-peso.txt'
    elif passedParameter == 'tipo':
        fileName = 'task3_' + modelName + '_' + strategyName + '_coppie_tipo-peso.txt'

    # controlliamo se il file esiste o no
    file = open('./outputfilesecondaparte/' + fileName,'w+')

    with open('./outputfilesecondaparte/' + fileName, 'w') as convert_file:
        convert_file.write(json.dumps(dictionaryFeaturesLatenti, indent=4, sort_keys=True))

In [5]:
# funzione usata per ottenre le top-k semantiche latenti.
# parametri della funzione:
# modelName: nome del modello da usare (ColorMoments, LBP, HOG)
# X: il tipo dell'immagine
# k: numero delle semantiche da estrarre
# strategyName: nome della tecnica con cui su vuole effettuare la riduzione di dimensionalità (PCA, SVD, LDA)

def ottieniKSemanticheLatenti(modelName, param, k, strategyName, passedParameter):
    # apriamo l'immagine

    # imagePath = 'secondaparte/images/image-' + X + '-1-1.png'
    # image = Image.open(imagePath)

    # # convertiamo l'immagine in un numpy array
    # data = asarray(image)
    # print(data)

    # plt.imshow(image, cmap ="gray")
    # plt.show()

    filtered_dir_list = []

    if passedParameter == 'X':
        filtered_dir_list = list(filter(lambda k: param in k, dir_list))
    elif passedParameter == 'Y':
        for i in range(len(dir_list)):
            splittedString = re.split('-', dir_list[i])

            if splittedString[2] == param:
                filtered_dir_list.append(dir_list[i])

    print("Numero di file su cui applichiamo i metodi: ", len(filtered_dir_list))
    #print(filtered_dir_list)
    dictionaryOfDescriptors = {}

    match modelName:
        case 'ColorMoments':
            print("SONO IN COLOR MOMENTS")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_ColorMoments.json')
            
            # returns JSON object as
            # a dictionary
            data = json.load(f)
            
            # Iterating through the json
            # list
            for key in data:
                #print(key)
                if key in filtered_dir_list:
                    dictionaryOfDescriptors[key] = data[key]
            
            # Closing file
            f.close()

        case 'LBP':
            print("SONO IN LBP")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_LBP.json')
            
            # returns JSON object as
            # a dictionary
            data = json.load(f)
            
            # Iterating through the json
            # list
            for key in data:
                #print(key)
                if key in filtered_dir_list:
                    dictionaryOfDescriptors[key] = data[key]
            
            # Closing file
            f.close()

        case 'HOG':
            
            print("HOG")

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_HOG.json')
            
            # returns JSON object as
            # a dictionary
            data = json.load(f)
            
            # Iterating through the json
            # list
            for key in data:
                #print(key)
                if key in filtered_dir_list:
                    dictionaryOfDescriptors[key] = data[key]
            
            # Closing file
            f.close()

        case _:
            return 4
            
    matrixObjFeatures = []

    lista = []

    for key in dictionaryOfDescriptors:
        matrixObjFeatures.append(dictionaryOfDescriptors[key])
        lista.append(key)

    print("Lunghezza di un descrittore: ", len(matrixObjFeatures[0]))
    print("Numero di oggetti: ", len(matrixObjFeatures))
    
    # calcoliamo il numero massimo di features latenti possibili, ovvero il minimo tra il numero di oggetti e di features di partenza
    numberOfFeatures = min(len(matrixObjFeatures[0]), len(matrixObjFeatures))

    match strategyName:
        case 'PCA':
            print("SONO IN PCA")

            numpyMatrixObjFeatures = np.array(matrixObjFeatures)

            covMatr = np.cov(numpyMatrixObjFeatures.T)

            print("\nNumero righe matrice di covarianza: ", len(covMatr))
            print("Numero colonne matrice di covarianza: ", len(covMatr[0]))
            print("La matrice di covarianza è quadrata? ", len(covMatr) == len(covMatr[0]))


            #TODO: RICONTROLLARE BENE
            pca = PCA(n_components = numberOfFeatures)
            converted_data = pca.fit_transform(matrixObjFeatures)
            print("\n")
            print(converted_data)

            print("\nNumero righe matrice di converted_data: ", len(converted_data))
            print("Numero colonne matrice di converted_data: ", len(converted_data[0]))

            # Questa imlementazione di PCA mi dà una matrice che ha tante righe quanti sono gli oggetti e tante colonne quante sono le features
            # mi descrive gli oggetti sulla base delle nuove features latenti

            createDictionaryAndWriteOnFile(converted_data, lista, passedParameter, strategyName, modelName)

            return 1

        case 'SVD':

            print("SONO IN SVD")

            image = np.array(matrixObjFeatures)

            # TODO: capire se dobbiamo passare già qua il parametro k
            u, s, vt = randomized_svd(image, n_components=numberOfFeatures)

            print("\nNumero righe di U: ", len(u))
            print("\nNumero colonne di U: ", len(u[0]))

            matrixS = np.diag(s)
            print("\nNumero righe di S: ", len(matrixS))
            print("\nNumero colonne di S: ", len(matrixS[0]))

            print("\nNumero righe di VT: ", len(vt))
            print("\nNumero colonne di VT: ", len(vt[0]))

            print("\nLeft Singular Vectors:")
            print(u)
            print("\nSingular Values:")
            print(np.diag(s))
            print("\nRight Singular Vectors:")
            print(vt)

            # SVD
            # U, s, VT = svd(image, full_matrices=False) # full_matrices=False
            # print("STAMPO U che ha lunghezza: ", len(U))

            # print("STAMPO U dove un oggetto ha lunghezza: ", len(U[0]))
            # print(U)

            # print("STAMPO S dove un oggetto ha lunghezza: ", len(s))
            # print(s)

            # print("STAMPO V trasposta dove un oggetto ha lunghezza: ", len(VT[0]))
            # print(VT)

            # print("STAMPO A")

            # Traspongo la matrice U per poter iterare sulle righe e quindi costruire la sequenza di coppie soggetto-peso per ogni feature latente

            createDictionaryAndWriteOnFile(u, lista, passedParameter, strategyName, modelName)

            return 2

        case 'LDA':
            print("SONO IN LDA")

            numpyMatrixObjFeatures = np.array(matrixObjFeatures)
            lda = LatentDirichletAllocation(n_components=numberOfFeatures,
                random_state=0)
            #lda.fit(numpyMatrixObjFeatures)
            #LatentDirichletAllocation(...)
            # get topics for some given samples:
            result = lda.fit_transform(numpyMatrixObjFeatures)
            print(result)
            print("LUNGHEZZA RESULT")
            print("righe: ", len(result))
            print("colonne: ", len(result[0]))

            # capire come fare LDA
            # # Define the number of topics or components
            # num_components=5

            # # Create LDA object
            # model=LatentDirichletAllocation(n_components=num_components)

            # # Fit and Transform SVD model on data
            # lda_matrix = model.fit_transform(matrixObjFeatures)

            # # Get Components 
            # lda_components=model.components_

            # for index, component in enumerate(lda_components):
            #     print(index)
            #     print(component)

            print("FINISCO LDA")

            #TODO: CAPIRE COME INTERPRETARE QUESTA MATRICE RESULT DA LDA
            createDictionaryAndWriteOnFile(result, lista, passedParameter, strategyName, modelName)

            print("FINISCO SCRITTURA SU FILE")

            # # define transform
            # lda = LinearDiscriminantAnalysis()
            # # prepare transform on dataset
            # lda.fit(matrixObjFeatures)
            # # apply transform to dataset
            # transformed = lda.transform(matrixObjFeatures)

            # print("transformed")
            # print(transformed)

            # # # define the pipeline
            # # steps = [('lda', LinearDiscriminantAnalysis()), ('m', GaussianNB())]

            # # # It can also be a good idea to standardize data prior to performing the LDA transform if the input variables have differing units or scales; for example:
            # # #steps = [('s', StandardScaler()), ('lda', LinearDiscriminantAnalysis()), ('m', GaussianNB())]
            # # model = Pipeline(steps=steps)
            return 3

        case _:
            return 4


# Funzione per task 3 e 4

In [6]:
# funzione usata per ottenre le top-k semantiche latenti.
# parametri della funzione:
# modelName: nome del modello da usare (ColorMoments, LBP, HOG)
# X: il tipo dell'immagine
# k: numero delle semantiche da estrarre
# strategyName: nome della tecnica con cui su vuole effettuare la riduzione di dimensionalità (PCA, SVD, LDA)

def ottieniKSemanticheLatentiTask34(modelName, param, k, strategyName):

    dictionaryOfDescriptors = {}

    match modelName:
        case 'ColorMoments':
            print("SONO IN COLOR MOMENTS")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_ColorMoments.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'LBP':
            print("SONO IN LBP")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_LBP.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'HOG':
            
            print("HOG")

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_HOG.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case _:
            return 4
            
    lista = []
    #dictionaryOfDescriptors contiene TUTTI i descrittori di TUTTI gli oggetti
    if param == 'tipo':
        lista = ['cc', 'con', 'emboss', 'jitter', 'neg', 'noise01', 'noise02', 'original', 'poster', 'rot', 'smooth', 'stipple']
        print(lista)

    elif param == 'soggetto':
        lista = []
        for i in range(40):
            lista.append(str(i)) #aggiungo il soggetto

        print(lista)
    else: 
        raise Exception("Param ha un valore sbagliato")
        
    arrayDiCentroidi = []

    filteredFile = []
    for i in range(len(lista)):
        filteredDictionary = {}
        for key in dictionaryOfDescriptors:
            if lista[i] in key:
                filteredFile.append(key)
                filteredDictionary[key] = dictionaryOfDescriptors[key]
        
        matrix = []
        #TODO: calcolare il centroide
        for key in filteredDictionary:
            matrix.append(filteredDictionary[key])
        
        npMatrix = np.array(matrix)

        centroide = np.mean(npMatrix, axis=0)
        arrayDiCentroidi.append(centroide)

    print(arrayDiCentroidi)
    print(len(arrayDiCentroidi[0]))

    #ciclare tutti i centroidi e calcolare le distanze tra di essi

    matrixForDecomposition = np.eye(len(arrayDiCentroidi)) #matrice tipo-tipo o soggetto-soggetto

    for i in range(len(arrayDiCentroidi)):
        for j in range(i, len(arrayDiCentroidi)):
            similarity = cosine_similarity(arrayDiCentroidi[i].reshape(1,-1), arrayDiCentroidi[j].reshape(1,-1)) # calcolo la similarità tra i due descrittori
            matrixForDecomposition[i][j] = similarity
            matrixForDecomposition[j][i] = similarity

    print(matrixForDecomposition)
    print("QUA 0")

    numpyMatrixForDecomposition = np.array(matrixForDecomposition)
    print("QUA 1")

    # calcoliamo il numero massimo di features latenti possibili, ovvero il minimo tra il numero di oggetti e di features di partenza
    numberOfFeatures = min(len(numpyMatrixForDecomposition[0]), len(numpyMatrixForDecomposition))
    
    # print("FILTERED FILE LEN: ", len(filteredFile))
    # print(filteredFile)

    print("PRIMA DELLO SWITCH")

    match strategyName:
        case 'PCA':
            print("SONO IN PCA")

            covMatr = np.cov(numpyMatrixForDecomposition.T)

            print("\nNumero righe matrice di covarianza: ", len(covMatr))
            print("Numero colonne matrice di covarianza: ", len(covMatr[0]))
            print("La matrice di covarianza è quadrata? ", len(covMatr) == len(covMatr[0]))

            #TODO: CONTROLLARE BENEEEE
            pca = PCA(n_components = numberOfFeatures)
            converted_data = pca.fit_transform(numpyMatrixForDecomposition)
            print("\n")
            print(converted_data)

            print("\nNumero righe matrice di converted_data: ", len(converted_data))
            print("Numero colonne matrice di converted_data: ", len(converted_data[0]))

            # Questa imlementazione di PCA mi dà una matrice che ha tante righe quanti sono gli oggetti e tante colonne quante sono le features
            # mi descrive gli oggetti sulla base delle nuove features latenti

            createDictionaryAndWriteOnFileTask34(converted_data, lista, param, strategyName, modelName)

            return 1

        case 'SVD':

            print("SONO IN SVD")

            image = np.array(numpyMatrixForDecomposition)

            # TODO: capire se dobbiamo passare già qua il parametro k
            u, s, vt = randomized_svd(image, n_components=numberOfFeatures)

            print("\nNumero righe di U: ", len(u))
            print("\nNumero colonne di U: ", len(u[0]))

            matrixS = np.diag(s)
            print("\nNumero righe di S: ", len(matrixS))
            print("\nNumero colonne di S: ", len(matrixS[0]))

            print("\nNumero righe di VT: ", len(vt))
            print("\nNumero colonne di VT: ", len(vt[0]))

            print("\nLeft Singular Vectors:")
            print(u)
            print("\nSingular Values:")
            print(np.diag(s))
            print("\nRight Singular Vectors:")
            print(vt)

            # SVD
            # U, s, VT = svd(image, full_matrices=False) # full_matrices=False
            # print("STAMPO U che ha lunghezza: ", len(U))

            # print("STAMPO U dove un oggetto ha lunghezza: ", len(U[0]))
            # print(U)

            # print("STAMPO S dove un oggetto ha lunghezza: ", len(s))
            # print(s)

            # print("STAMPO V trasposta dove un oggetto ha lunghezza: ", len(VT[0]))
            # print(VT)

            # print("STAMPO A")

            # Traspongo la matrice U per poter iterare sulle righe e quindi costruire la sequenza di coppie soggetto-peso per ogni feature latente

            createDictionaryAndWriteOnFileTask34(u, lista, param, strategyName, modelName)

            return 2

        case 'LDA':
            print("SONO IN LDA")

            numpyMatrixObjFeatures = np.array(numpyMatrixForDecomposition)
            lda = LatentDirichletAllocation(n_components=numberOfFeatures,
                random_state=0)
            
            result = lda.fit_transform(numpyMatrixObjFeatures)
            print(result)
            print("LUNGHEZZA RESULT")
            print("righe: ", len(result))
            print("colonne: ", len(result[0]))


            # capire come fare LDA
            # # Define the number of topics or components
            # num_components=5

            # # Create LDA object
            # model=LatentDirichletAllocation(n_components=num_components)

            # # Fit and Transform SVD model on data
            # lda_matrix = model.fit_transform(matrixObjFeatures)

            # # Get Components 
            # lda_components=model.components_

            # for index, component in enumerate(lda_components):
            #     print(index)
            #     print(component)

            print("FINISCO LDA")

            createDictionaryAndWriteOnFileTask34(result, lista, param, strategyName, modelName)
            print("FINISCO SCRITTURA SU FILE")


            # # define transform
            # lda = LinearDiscriminantAnalysis()
            # # prepare transform on dataset
            # lda.fit(matrixObjFeatures)
            # # apply transform to dataset
            # transformed = lda.transform(matrixObjFeatures)

            # print("transformed")
            # print(transformed)

            # # # define the pipeline
            # # steps = [('lda', LinearDiscriminantAnalysis()), ('m', GaussianNB())]

            # # # It can also be a good idea to standardize data prior to performing the LDA transform if the input variables have differing units or scales; for example:
            # # #steps = [('s', StandardScaler()), ('lda', LinearDiscriminantAnalysis()), ('m', GaussianNB())]
            # # model = Pipeline(steps=steps)
            return 3

        case _:
            raise Exception("Metodo di decomposizione non valido")


> **Soluzione scelta:** Per ottenere le matrici di similarità tipo-tipo o soggetto-soggetto abbiamo recuperato tutte le immagini di quel tipo o di quel soggetto a seconda del caso e abbiamo calcolato per ogni tipo o soggetto un centroide.
Grazie a questi centroidi abbiamo poi costruito la matrice di similarità sfruttando la cosine similarity.

> **Nota bene:** Questa soluzione implementata per costruire la matrice di similarità è solo una delle tante disponibili. Avremmo potuto usare i medoidi oppure costruire una matrice per ogni tipo o soggetto e poi confrontare nella maniera corretta (perchè il confronto di matrici è soggetto all'ordine degli oggetti in esse) le matrici e calcolarne la distanza/similarit.

# Funzione per task 5

In [7]:
def getNsimilarImages(imageQueryFile, latentSemanticFileName):

    done = False
    try:
        # apriamo l'immagine
        image = Image.open('./secondaparte/images/' + imageQueryFile)
        file = open(latentSemanticFileName)
        done = True
    except:
        print("Il file di query e/o quello delle semantiche latenti non esiste! Inserire un nome corretto!")


    if done:
        # convertiamo l'immagine in un numpy array
        data = asarray(image)
        print(data)


# Funzione per task 8

In [8]:
# funzione usata per ottenre le top-k semantiche latenti.
# parametri della funzione:
# modelName: nome del modello da usare (ColorMoments, LBP, HOG)
# X: il tipo dell'immagine
# k: numero delle semantiche da estrarre
# strategyName: nome della tecnica con cui su vuole effettuare la riduzione di dimensionalità (PCA, SVD, LDA)

def createSimilarityMatrix(modelName, param):

    dictionaryOfDescriptors = {}

    match modelName:
        case 'ColorMoments':
            print("SONO IN COLOR MOMENTS")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_ColorMoments.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'LBP':
            print("SONO IN LBP")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_LBP.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'HOG':
            
            print("HOG")

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_HOG.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case _:
            return 4
            
    lista = []
    #dictionaryOfDescriptors contiene TUTTI i descrittori di TUTTI gli oggetti
    if param == 'tipo':
        lista = ['cc', 'con', 'emboss', 'jitter', 'neg', 'noise01', 'noise02', 'original', 'poster', 'rot', 'smooth', 'stipple']
        print(lista)

    elif param == 'soggetto':
        lista = []
        for i in range(40):
            lista.append(str(i)) #aggiungo il soggetto

        print(lista)
    else: 
        raise Exception("Param ha un valore sbagliato")
        
    arrayDiCentroidi = []

    filteredFile = []
    for i in range(len(lista)):
        filteredDictionary = {}
        for key in dictionaryOfDescriptors:
            if lista[i] in key:
                filteredFile.append(key)
                filteredDictionary[key] = dictionaryOfDescriptors[key]
        
        matrix = []
        #TODO: calcolare il centroide
        for key in filteredDictionary:
            matrix.append(filteredDictionary[key])
        
        npMatrix = np.array(matrix)

        centroide = np.mean(npMatrix, axis=0)
        arrayDiCentroidi.append(centroide)

    print(arrayDiCentroidi)
    print(len(arrayDiCentroidi[0]))

    #ciclare tutti i centroidi e calcolare le distanze tra di essi

    matrixForDecomposition = np.eye(len(arrayDiCentroidi)) #matrice tipo-tipo o soggetto-soggetto

    for i in range(len(arrayDiCentroidi)):
        for j in range(i, len(arrayDiCentroidi)):
            similarity = cosine_similarity(arrayDiCentroidi[i].reshape(1,-1), arrayDiCentroidi[j].reshape(1,-1)) # calcolo la similarità tra i due descrittori
            matrixForDecomposition[i][j] = similarity
            matrixForDecomposition[j][i] = similarity

    print(matrixForDecomposition)
    
    return matrixForDecomposition

# Page Rank Algorithm

Link: https://www.geeksforgeeks.org/page-rank-algorithm-implementation/

In [9]:
def pagerank(G, alpha=0.85, personalization=None,
			max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
			dangling=None):
	"""Return the PageRank of the nodes in the graph.

	PageRank computes a ranking of the nodes in the graph G based on
	the structure of the incoming links. It was originally designed as
	an algorithm to rank web pages.

	Parameters
	----------
	G : graph
	A NetworkX graph. Undirected graphs will be converted to a directed
	graph with two directed edges for each undirected edge.

	alpha : float, optional
	Damping parameter for PageRank, default=0.85.

	personalization: dict, optional
	The "personalization vector" consisting of a dictionary with a
	key for every graph node and nonzero personalization value for each node.
	By default, a uniform distribution is used.

	max_iter : integer, optional
	Maximum number of iterations in power method eigenvalue solver.

	tol : float, optional
	Error tolerance used to check convergence in power method solver.

	nstart : dictionary, optional
	Starting value of PageRank iteration for each node.

	weight : key, optional
	Edge data key to use as weight. If None weights are set to 1.

	dangling: dict, optional
	The outedges to be assigned to any "dangling" nodes, i.e., nodes without
	any outedges. The dict key is the node the outedge points to and the dict
	value is the weight of that outedge. By default, dangling nodes are given
	outedges according to the personalization vector (uniform if not
	specified). This must be selected to result in an irreducible transition
	matrix (see notes under google_matrix). It may be common to have the
	dangling dict to be the same as the personalization dict.

	Returns
	-------
	pagerank : dictionary
	Dictionary of nodes with PageRank as value

	Notes
	-----
	The eigenvector calculation is done by the power iteration method
	and has no guarantee of convergence. The iteration will stop
	after max_iter iterations or an error tolerance of
	number_of_nodes(G)*tol has been reached.

	The PageRank algorithm was designed for directed graphs but this
	algorithm does not check if the input graph is directed and will
	execute on undirected graphs by converting each edge in the
	directed graph to two edges.

	
	"""
	if len(G) == 0:
		return {}

	if not G.is_directed():
		D = G.to_directed()
	else:
		D = G

	# Create a copy in (right) stochastic form
	W = nx.stochastic_graph(D, weight=weight)
	N = W.number_of_nodes()

	# Choose fixed starting vector if not given
	if nstart is None:
		x = dict.fromkeys(W, 1.0 / N)
	else:
		# Normalized nstart vector
		s = float(sum(nstart.values()))
		x = dict((k, v / s) for k, v in nstart.items())

	if personalization is None:

		# Assign uniform personalization vector if not given
		p = dict.fromkeys(W, 1.0 / N)
	else:
		missing = set(G) - set(personalization)
		if missing:
			raise nx.NetworkXError('Personalization dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(personalization.values()))
		p = dict((k, v / s) for k, v in personalization.items())

	if dangling is None:

		# Use personalization vector if dangling vector not specified
		dangling_weights = p
	else:
		missing = set(G) - set(dangling)
		if missing:
			raise nx.NetworkXError('Dangling node dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(dangling.values()))
		dangling_weights = dict((k, v/s) for k, v in dangling.items())
	dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

	# power iteration: make up to max_iter iterations
	for _ in range(max_iter):
		xlast = x
		x = dict.fromkeys(xlast.keys(), 0)
		danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
		for n in x:

			# this matrix multiply looks odd because it is
			# doing a left multiply x^T=xlast^T*W
			for nbr in W[n]:
				x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
			x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

		# check convergence, l1 norm
		err = sum([abs(x[n] - xlast[n]) for n in x])
		if err < N*tol:
			return x
	raise nx.NetworkXError('pagerank: power iteration failed to converge '
						'in %d iterations.' % max_iter)

